In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from uncertainties import ufloat
# from uncertainties.umath import *
# from uncertainties import unumpy
from uncertainties.unumpy import uarray

################################
# FIRST DEFINE GLOBAL CONSTANTS
################################
g = 9.81
mu0 = 1.25663706 * 10**(-6)


################################
# IMPORT DATA TO NP & PD
################################

'''
# into np
with open('data/magnetic.csv') as file:
    np_data = np.genfromtxt(file, delimiter=',')
'''


# into pandas
xl = pd.ExcelFile('data/magnetic.xlsx')
df = xl.parse('Sheet1')


################################
# MAGNETIC FIELD DATA
################################

b_cal_length = 10.00 / 1000

vile_weight = df['vile_weights'].dropna().apply(lambda x: ufloat(x, .000001))
vile_weight = vile_weight.sum() / len(vile_weight)

# magnet masses from calibration with and without the current turned on
# you can apply the error with a lambda function or uarray (see a few lines down)
b_cal_scale_no_current = df['scale_no_mag_si'].dropna().apply(lambda x: ufloat(x, .000001))
b_cal_scale_with_current = df['scale_with_mag_si'].dropna().apply(lambda x: ufloat(x, .000001))


# current error according to the manufacturer's specs
current_error = 1000 * df['current'].dropna() * 10 ** -6 + 3 * 15 * 10**-6

# currents used during calibration
# this method returns a np array, so I convert it back into a pd element
b_cal_current = pd.Series(uarray(df['current'].dropna(), current_error))

b_cal_mass_difference = b_cal_scale_no_current - b_cal_scale_with_current
# print(b_cal_mass_difference)


################################
# MAGNETIC FIELD EQUATION
################################

# define the magnetic field as in lecture
def mag_field(mass, current, length):
    return (mass * g) / (current * length)


bcal = mag_field(b_cal_mass_difference, b_cal_current, b_cal_length)

################################
# MAGNETIC FIELD FUNCTION FIT
################################

# separate the values and error
b_cal_values = bcal.apply(lambda x: x.n)
b_cal_error = bcal.apply(lambda x: x.s)

# same thing for the current
current_values = b_cal_current.apply(lambda x: x.n)


def lin_fit(x, a, b):
    return a*x + b


# propagating error for model fits is outside the scope of this course
popt, pcov = curve_fit(lin_fit, current_values, b_cal_values)
b_fit = lin_fit(current_values, *popt)


################################
# PLOT THE RESULTS
################################
plt.errorbar(current_values,
             b_cal_values,
             yerr=b_cal_error,
             fmt='o',
             label='B Data')

plt.plot(current_values, b_fit, label='B Fit')
plt.ylim(.4, .43)
plt.legend()
plt.xlabel('Current Value (A)')
plt.ylabel('Magnetic Field (T)')
plt.savefig('figures/figure1.png')
# plt.show()

r_i = b_cal_values - b_fit
plt.clf()
plt.errorbar(
    current_values,
    r_i,
    yerr=b_cal_error,
    fmt='o')
plt.xlabel('Current Value (A)')
plt.ylabel('Residuals (T)')
plt.savefig('figures/figure1b.png')
# plt.show()


################################
# Xm DATA
################################

# let's go ahead and condense the b field down to a single point
bcal = bcal.sum() / len(bcal)

# strip whitespace off sample names
name = df['sample_name'].str.strip()

# density values are literature and have ~0 uncertainty
density = df['density_kg_m']
lit_xm = df['lit_xm']

# propagating uncertainty for all data we collected
area = \
        df['area_param_1'].apply(lambda x: ufloat(x, .0001)) * \
        df['area_param_2'].apply(lambda x: ufloat(x, .0001))


'''
the area uncertainty for cobalt is very close to zero, and we are given the
dimesions of the cobalt wire - so I will assume that the dimensions we are
given are exact. Without this assumption, it's really easy to hit an error bar
of 80,000
'''
area[4] = area[4].n

height = df['height_sample'].apply(lambda x: ufloat(x, .0001))
real_mass = df['sample_mass_si'].apply(lambda x: ufloat(x, .000001)) - vile_weight
volume = area * height
theory_mass = volume * density

# ensure that there are no percentages over 100 without dropping uncertainties
percent_real = (real_mass / theory_mass).apply(lambda x: ufloat(min(abs(x.n), 1), x.s))
percent_real[4] = percent_real[4].n
print(percent_real)

# update area so that it represents the actual area occupied by the substance
area = area * percent_real


no_sample_magnet_mass = df['scale_mass_si'].apply(lambda x: ufloat(x, .000001))
sample_magnet_mass = df['sample_magnet_si'].apply(lambda x: ufloat(x, .000001))
sample_mass_difference = no_sample_magnet_mass - sample_magnet_mass


################################
# Xm FUNCTION
################################
def xm(mass_difference, area):
    return (2 * mu0 * mass_difference * g) / (area * bcal**2)


sample_xm = xm(sample_mass_difference, area)
sample_xm_values = sample_xm.apply(lambda x: x.n)
sample_xm_err = sample_xm.apply(lambda x: x.s)

################################
# PLOTTING DATA
################################
plt.clf()

plt.bar(name[0:4], sample_xm_values[0:4], yerr=sample_xm_err[0:4],
        label='Data', alpha=.5)

plt.bar(name[0:4], lit_xm[0:4], label=' Lit Value (If Exists)', alpha=.4)
plt.ylabel('Volume X_m in SI')
plt.legend()
plt.savefig('figures/figure2')
# plt.show()

plt.clf()
plt.bar(name[[5,6,9,10,11]], sample_xm_values[[5,6,9,10,11]],
        yerr=sample_xm_err[[5,6,9,10,11]], alpha=.5, label='Data')

plt.bar(name[[5,6,9,10,11]], lit_xm[[5,6,9,10,11]], label=' Lit Value (If Exists)', alpha=.4)
plt.ylabel('Volume X_m in SI')
plt.legend()
plt.savefig('figures/figure3')
# plt.show()

plt.clf()
plt.bar(name[[7,8,13,14]], sample_xm_values[[7,8,13,14]],
        yerr=sample_xm_err[[7,8,13,14]],label='Data')
plt.bar(name[[7,8,13,14]], lit_xm[[7,8,13,14]], label=' Lit Value (If Exists)', alpha=.4)
plt.ylabel('Volume X_m in SI')
plt.legend()
plt.savefig('figures/figure4')
# plt.show()

plt.clf()
plt.bar(name[[15,17]], sample_xm_values[[15,17]],
        yerr=sample_xm_err[[15,17]], label='Data')

plt.bar(name[[15,17]], lit_xm[[15,17]], label=' Lit Value (If Exists)', alpha=.4)
plt.ylabel('Volume X_m in SI')
plt.legend()
plt.savefig('figures/figure5')
# plt.show()


plt.clf()
plt.bar(name[[12,16,18]], sample_xm_values[[12,16,18]],
        yerr=sample_xm_err[[12,16,18]], label='Data')

plt.bar(name[[12,16,18]], lit_xm[[12,16,18]], label=' Lit Value (If Exists)', alpha=.4)
plt.ylabel('Volume X_m in SI')
plt.legend()
plt.savefig('figures/figure6')
# plt.show()

plt.clf()
plt.bar(name[[4]], sample_xm_values[[4]], yerr=sample_xm_err[[4]], label='Data')
plt.bar(name[[4]], lit_xm[[4]], label=' Lit Value (If Exists)', alpha=.4)
plt.ylabel('Volume X_m in SI')
plt.legend()
plt.savefig('figures/figure7')
# plt.show()


plt.clf()
x = np.linspace(-.001, .0175)
y = x
plt.plot(x, y, label='y = x (desired result)')

plt.errorbar(lit_xm, sample_xm_values, fmt='.', yerr=sample_xm_err, label='Data Points')
plt.xlabel('Literature X_m in SI (Unitless)')
plt.ylabel('Measured X_m (Unitless)')

plt.legend()
plt.savefig('figures/figure8')
plt.show()
print(sample_xm)

'''
This labels all the points on the scatterplot... but it gets loud with this
data
# plot_df = pd.concat([name, lit_xm, sample_xm_values], axis=1)
# plot_df.columns = ['name', 'lit_xm', 'sample_xm_values']

# for i, point in plot_df.iterrows():
    # plt.annotate(str(point['name']), (point['lit_xm'], point['sample_xm_values']))

# plt.xlim(-.001, 0.0001)
# plt.ylim(-.001, 0.000)

# plt.xlim(-.00005, 0.0006)
# plt.ylim(-.000, 0.00005)
# plt.savefig('figures/figure9')

# plt.xlim(-.00005, 0.0006)
# plt.ylim(.000, .001)
# plt.savefig('figures/figure9')
# plt.show()
'''


0     1.000+/-0.015
1     1.000+/-0.016
2     1.000+/-0.015
3     0.569+/-0.008
4                 1
5       1.00+/-0.30
6     0.348+/-0.005
7     0.255+/-0.004
8     0.336+/-0.005
9     0.633+/-0.009
10    0.551+/-0.008
11    0.630+/-0.009
12    0.575+/-0.008
13    0.384+/-0.006
14    0.534+/-0.008
15    0.363+/-0.005
16    0.991+/-0.014
17            1+/-7
18    0.552+/-0.008
dtype: object


0       (-1.41+/-0.22)e-05
1        (1.57+/-0.23)e-05
2      0.000180+/-0.000005
3     -0.000160+/-0.000005
4            2.236+/-0.018
5       -0.00086+/-0.00028
6      0.000750+/-0.000016
7        0.01289+/-0.00027
8          0.0217+/-0.0004
9      0.000755+/-0.000016
10     0.000699+/-0.000015
11     0.000144+/-0.000004
12         (6.1+/-0.4)e-05
13       0.00450+/-0.00009
14       0.00219+/-0.00005
15       0.00379+/-0.00008
16       (2.40+/-0.21)e-05
17             0.01+/-0.13
18         (3.3+/-0.4)e-05
dtype: object


"\nThis labels all the points on the scatterplot... but it gets loud with this\ndata\n# plot_df = pd.concat([name, lit_xm, sample_xm_values], axis=1)\n# plot_df.columns = ['name', 'lit_xm', 'sample_xm_values']\n\n# for i, point in plot_df.iterrows():\n    # plt.annotate(str(point['name']), (point['lit_xm'], point['sample_xm_values']))\n\n# plt.xlim(-.001, 0.0001)\n# plt.ylim(-.001, 0.000)\n\n# plt.xlim(-.00005, 0.0006)\n# plt.ylim(-.000, 0.00005)\n# plt.savefig('figures/figure9')\n\n# plt.xlim(-.00005, 0.0006)\n# plt.ylim(.000, .001)\n# plt.savefig('figures/figure9')\n# plt.show()\n"

In [19]:
pretty_df = pd.DataFrame()
pretty_df['Sample Name'] = name
pretty_df['Volume X_m in SI'] = sample_xm.apply(lambda x: x.n)
pretty_df['X_m Uncertainty'] = sample_xm.apply(lambda x: x.s)
pretty_df['Literature Volume X_m Values'] = lit_xm

In [20]:
pretty_df

,Sample Name,Volume X_m in SI,X_m Uncertainty,Literature Volume X_m Values
0,Cu_110_Alloy,-0.000014,0.000002,-0.000010
1,Al_1100_Alloy,0.000016,0.000002,0.000021
2,Ti_Grade_2,0.000180,0.000005,0.000181
3,Bi_Pellets,-0.000160,0.000005,-0.000170
4,co,2.236249,0.018296,NaN
5,grap,-0.000864,0.000282,-0.000800
6,NdCI_3,0.000750,0.000016,NaN
7,Gd_2_O3,0.012894,0.000266,0.013666
8,Er_2_O_3,0.021730,0.000448,0.020982
9,NH42FeSO46H2O,0.000755,0.000016,NaN
